<a href="https://colab.research.google.com/github/esragcetnky/Stroke-Prediction/blob/main/Stroke_Prediction_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Import Libraries

In [ ]:
# at first these 3 libraries are enough,
# when trying to build model, i will add other necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# let's read our data
data= pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

# 2.Analyzing and Understanding Dataset

* let's put data to dataframe

In [ ]:
dataframe=pd.DataFrame(data)

In [ ]:
dataframe.shape

* So we have 12 features and 5110 samples in our datasets.
* One of the 12 features is the label

Let's see the names of features

In [ ]:
dataframe.columns

In [ ]:
# let's see the top 5 sample, so we can understand dataset better
dataframe.head()

# 3.Visualize Dataset

* visualizing data is really helpful
* it can give better understanding relations between features


Matplotlib library will be used for visualization

First, i wanna check on distributions

In [ ]:
plt.style.use('seaborn-darkgrid')

## *Histogram Plot*

In [ ]:
fig, (ax0,ax1,ax2)=plt.subplots(nrows=1,
                                ncols=3,
                                figsize=(30,10))

# AGE DISTRIBUTION 
ax0.hist(dataframe['age'],
         bins=15,
         color='blue')
ax0.set_ylim([200,500])
ax0.set_xlim([0,80])
ax0.set_xlabel('Age')
ax0.set_title('Age distribution')

# AVG GLUCOSE LEVEL DISTRIBUTION
ax1.hist(dataframe['avg_glucose_level'],
         bins=25)
ax1.set_xlabel('Average Glucose Level')
ax1.set_xlim([50,250])
ax1.set_title('Average Glucose Level distribution')

# BMI DISTRIBUTION
ax2.hist(dataframe['bmi'],
         bins=25,
         color='green')
ax2.set_xlabel('Bmi')
ax2.set_ylim([0,1000])
ax2.set_xlim([10,60])
ax2.set_title('Bmi distribution')

fig.show()

## *Pie Chart*

In [ ]:
fig, ((ax0,ax1),(ax2,ax3))=plt.subplots(nrows=2,
                                       ncols=2,
                                       figsize=(15, 15))
def label_function(val):
    return f'{val / 100 * len(dataframe):.0f}\n{val:.0f}%'

# EVER MARRIED
dataframe.groupby('ever_married').size().plot(kind='pie', 
                                              autopct=label_function,
                                              textprops={'fontsize': 15},
                                              ax=ax0)
ax0.set_ylabel('Ever Married',size=15)

# GENDER
dataframe.groupby('gender').size().plot(kind='pie', 
                                      autopct=label_function, 
                                      textprops={'fontsize': 15},
                                      colors=['violet', 'lime','tomato'],
                                      ax=ax1)
ax1.set_ylabel('Per gender',size=15)

# SMOKING STATUS
dataframe.groupby('smoking_status').size().plot(kind='pie',
                                                 autopct=label_function, 
                                                 textprops={'fontsize': 15},
                                                 colors=['tomato', 'gold','violet','lime'],
                                                 ax=ax2)
ax2.set_ylabel('Smoking Status',size=15)

# RESIDENCE TYPE 
dataframe.groupby('Residence_type').size().plot(kind='pie',
                                                 autopct=label_function, 
                                                 textprops={'fontsize': 15},
                                                 ax=ax3)                                         
ax3.set_ylabel('Residence Type',size=15)

# showing the figure
fig.show()

# 4.Split and Formatting Data

## *Creating dummy variables*

* when training model, cant use string values so that is why i am creating dummy variables

In [ ]:
# creating dummy variables
def dummy_creation(dataset,dummy_categories):
  for i in dummy_categories:
    dataset_dummy=pd.get_dummies(dataset[i])
    dataset=pd.concat([dataset,dataset_dummy],
                      axis=1)
    dataset=dataset.drop(i,axis=1)
  return dataset

In [ ]:
dataframe=dummy_creation(dataframe, ['gender','ever_married','work_type','Residence_type','smoking_status'])

## *Dropping useless columns*


* i dont need id column, so dropped that column

In [ ]:
dataframe=dataframe.drop(columns='id')

In [ ]:
dataframe

* in gender feature, there is one sample with 'Other' value, i think dont need that value so i am gonna delete this sample

In [ ]:
dataframe[dataframe['Other']==1]

In [ ]:
dataframe =dataframe.drop(3116)

* now let's check

In [ ]:
dataframe[dataframe['Other']==1]

* perfect, now i can drop 'Other' column, i dont need that

In [ ]:
dataframe= dataframe.drop(columns='Other')

In [ ]:
dataframe.head()

## *Checking for null values*

* before split data, i wanna if there are null values

In [ ]:
dataframe.isnull().values.any(), dataframe.isnull().sum().sum()

* looks like there are 201 null values in the dataset, lets look which columns has null value

In [ ]:
count_nan=dataframe.isnull().sum()
print(count_nan)

In [ ]:
dataframe['bmi'].isnull().sum()

* I am dropping nan values, now we have 5110-201 sample

In [ ]:
dataframe.dropna(subset=['bmi'],
                 inplace=True)

In [ ]:
dataframe.isnull().values.any()

In [ ]:
dataframe.shape

## *Heatmap*

* i wanna check on colleration between feature using heatmap

In [ ]:
import seaborn as sns
fig, ax=plt.subplots(figsize=(20,20))
sns.heatmap(dataframe.corr(),
            annot=True,
            linewidth=.5,
            ax=ax)
fig.show()

## Splitting data as train and test dataset

* first, i will split target and data

In [ ]:
y=dataframe['stroke']

In [ ]:
y.shape

In [ ]:
y[y==1]

In [ ]:
x=dataframe.drop(columns='stroke')

In [ ]:
x.columns, x.shape

* i am going to split dataset using 'train_test_split' from sklearn

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

# 5.Build Model

## *Import libraries*

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Softmax,MaxPool2D

## *Creating models*

In [ ]:
model=Sequential()

## *Adding Layers*

In [ ]:
input_size=x_train.shape[1]
model.add(Dense(units=50,
                activation='relu',              
                input_shape=[input_size,]))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(units=50,
                activation='relu'))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(units=50,
                activation='relu'))

In [ ]:
model.add(Dense(units=1,
                activation='sigmoid'))

In [ ]:
model.summary()

## *Compile Model*

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='accuracy')

## *Train Model*

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=16,
          epochs=50)

## *Test Model*

In [ ]:
y_pred=model.predict(x_test).flatten()

In [ ]:
y_pred=np.round(y_pred)

In [ ]:
loss_value, accuracy_value= model.evaluate(x_test,y_test)

In [ ]:
print(f'Accuracy of the model: {accuracy_value}')
print(f'Loss of the model: {loss_value}')

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

In [ ]:
cf=confusion_matrix(y_test,y_pred)
print(cf)

## *Saving Model*

In [ ]:
model.save('/content/drive/My Drive/Stroke_Prediction')